# 60_binary — 이진형

In [ ]:

%run 00_config.ipynb
import pandas as pd, numpy as np

BINARY_VARS = [
    "q3_3_1","q3_3_2","q3_3_3","q3_3_4","q3_3_5","q3_3_6","q3_3_7","q3_3_8",
    "q3_3_9","q3_3_10","q3_3_11","q3_3_12","q3_3_13","q3_3_14","q3_3_15","q3_3_16",
    "DM1","DM7"
]

df = pd.read_excel(DATA_PATH)
w = choose_weight(df)

def wmean_binary(x, w):
    m = x.notna()
    wv = w[m]; xv = x[m].astype(float)
    tot = wv.sum()
    return float((wv*xv).sum()/tot) if tot>0 else np.nan

# 유병률
rows = []
for col in BINARY_VARS:
    if col not in df.columns: continue
    u = df[col].mean()*100
    wr = wmean_binary(df[col], w)*100
    rows.append({"variable": col, "unweighted_%(=1)": round(float(u),2) if pd.notna(u) else np.nan,
                 "weighted_%(=1)": round(float(wr),2) if pd.notna(wr) else np.nan})
bin_summary = pd.DataFrame(rows).sort_values("variable")

# 활동 수 합계
multi_cols = [c for c in BINARY_VARS if c.startswith("q3_3_") and c in df.columns]
df["activity_count"] = df[multi_cols].sum(axis=1) if multi_cols else np.nan

# 만족도와의 상관
rows = []
for target in ["q3","q7"]:
    if target not in df.columns: continue
    for col in BINARY_VARS:
        if col not in df.columns: continue
        m = df[col].notna() & df[target].notna()
        if m.sum()<3:
            r = np.nan
        else:
            r = np.corrcoef(df.loc[m, col].astype(float), df.loc[m, target].astype(float))[0,1]
        rows.append({"var": col, "target": target, "pearson_r": r, "n": int(m.sum())})
bin_corr = pd.DataFrame(rows).sort_values(["target","var"])

bin_summary.to_csv(OUT_DIR/"binary_summary.csv", index=False, encoding="utf-8-sig")
bin_corr.to_csv(OUT_DIR/"binary_vs_targets_corr.csv", index=False, encoding="utf-8-sig")
bin_summary.head(10)
